In [1]:
import numpy as np
import random
import sys
import io
import os
from mb_utils import *
%matplotlib inline
import pickle

import pandas as pd

/anaconda/lib/python3.6/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [9]:
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
from keras.utils import np_utils

In [2]:
from importlib import reload
import mb_utils

In [10]:
reload(mb_utils)
from mb_utils import *

In [4]:
Tx = 2754 #5511 # The number of time steps input to the model from the spectrogram
n_freq = 101 # Number of frequencies input to the model at each time step of the spectrogram
Ty = 1375 # The number of time steps in the output of our model

# model training

In [11]:
# load training features and labels
file = open('XY_train/X_100_5sec.pkl', 'rb')
X_training = pickle.load(file)
file.close()

file = open('XY_train/y_100_5sec.pkl', 'rb')
y_training = pickle.load(file)
file.close()

# load validation features and labels
file = open('XY_val/X_20_5sec.pkl', 'rb')
X_val = pickle.load(file)
file.close()

file = open('XY_val/y_20_5sec.pkl', 'rb')
y_val = pickle.load(file)
file.close()

# one-hot encode labels
y_training_oh = encode_and_oh_labels(y_training)
y_val_oh = encode_and_oh_labels(y_val)

In [12]:
label_decoder_dict = {0:'empty', 1:'enough', 2:'not_enough'}

In [25]:
# shuffle training data explicitly
np.random.seed(134)
perm = np.random.permutation(len(X_training))

X_training = X_training[perm]
y_training = y_training[perm]
y_training_oh = y_training_oh[perm]

In [93]:
def model(input_shape):
    X_input = Input(shape = input_shape)
    
    X = Conv1D(filters=196, kernel_size=15, strides=4)(X_input)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    #X = Dropout(0.0)(X)
    
    X = LSTM(units=128)(X)
    X = Dense(3, activation='softmax')(X)

    model = Model(inputs = X_input, outputs = X)
    
    return model  

In [94]:
model = model(input_shape = (Tx, n_freq))

In [95]:
#opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

In [97]:
model.fit(X_training[:40], y_training_oh[:40], batch_size=3, epochs=15, shuffle=True,
         validation_data = [X_val, y_val_oh])

Train on 40 samples, validate on 40 samples
Epoch 1/15
40/40 [==============================] - 12s 291ms/step - loss: 0.6252 - acc: 0.7750 - val_loss: 1.5570 - val_acc: 0.4250

In [20]:
# get predictions on validation set
preds = model.predict(X_val)
preds = [label_decoder_dict[v] for v in np.argmax(preds, axis=1)]

In [81]:
y_training[:20]

array(['not_enough', 'not_enough', 'empty', 'not_enough', 'empty', 'empty',
       'not_enough', 'not_enough', 'empty', 'empty', 'enough', 'empty',
       'not_enough', 'enough', 'empty', 'empty', 'empty', 'empty', 'empty',
       'empty'],
      dtype='<U10')

In [82]:
# get predictions on training set
preds = model.predict(X_training[:20])

In [83]:
preds

array([[  8.54240637e-03,   2.30323942e-03,   9.89154398e-01],
       [  5.26001334e-01,   3.57754737e-01,   1.16243944e-01],
       [  5.41951299e-01,   3.49923611e-01,   1.08125180e-01],
       [  4.30443019e-01,   3.57809573e-01,   2.11747453e-01],
       [  4.68633592e-01,   3.26568693e-01,   2.04797685e-01],
       [  9.80196059e-01,   1.92632398e-03,   1.78776402e-02],
       [  6.06514573e-01,   3.02205354e-01,   9.12800580e-02],
       [  9.54624861e-02,   5.30080535e-02,   8.51529539e-01],
       [  5.41039407e-01,   3.50230277e-01,   1.08730271e-01],
       [  4.10865039e-01,   1.77947193e-01,   4.11187738e-01],
       [  2.54908919e-01,   3.27823013e-01,   4.17268038e-01],
       [  6.44898236e-01,   5.96531443e-02,   2.95448661e-01],
       [  8.97464082e-02,   2.19169743e-02,   8.88336599e-01],
       [  4.97158080e-01,   3.72391224e-01,   1.30450651e-01],
       [  5.36056638e-01,   3.50246519e-01,   1.13696814e-01],
       [  9.98622775e-01,   3.96792078e-04,   9.8050327

In [84]:
preds_text = [label_decoder_dict[v] for v in np.argmax(preds, axis=1)]

In [85]:
preds_text

['not_enough',
 'empty',
 'empty',
 'empty',
 'empty',
 'empty',
 'empty',
 'not_enough',
 'empty',
 'not_enough',
 'not_enough',
 'empty',
 'not_enough',
 'empty',
 'empty',
 'empty',
 'not_enough',
 'empty',
 'empty',
 'not_enough']

In [86]:
val_results = pd.DataFrame({'labels':y_training[:20], 'preds':preds_text})

In [87]:
pd.crosstab(val_results['labels'], val_results['preds'])

preds,empty,not_enough
labels,,
empty,9,3
enough,1,1
not_enough,3,3
